# The Stupid Games I Play: Wordzee

Lately I've been playing this game called Wordzee a lot and have been wanting to do a mathematical analysis on it. I'm mainly interested in two things about this game: strategies & the most points you could have in a game. In this article, I will go over the latter expect another post about this game at some point in the future.

## How You Play The Game
Wordzee is a two-player turn-based word game, where the highest score wins. Each game has five rounds where you assemble the best word(s) possible from a set of random letters. If you don't like some letters, you can use the swap button to exchange some letters for some random ones. You have the option to hold some letters and can only swap up two times. For each word, choose the best place on the game board to play it. There are five rows you can place words in, as long as the word isn't longer than the row. Each tile has its own point value and certain squares and rows on the board will grant bonus points.


## Gathering Data
At this point, we could begin the analysis. But without data, the analysis this whole thing would remain symbolic, and I'd like to have some concrete numbers as we go along. The two questions that we need answered are:

1. What words are valid?
2. How many points is each letter worth?

I couldn't find a list of official words for the game, so I used the list for Scrabble. To get the points of the letters, I just played a bunch of games and took note of what the numbers were.

### Valid Words
I couldn't find an available list of Scrabble words as a dataset, so I decided to gather the data myself. I scrapped the words off the Scrabble section of the [Merriam-Webster](https://scrabble.merriam.com) website. The script I used for that can be found [here](https://www.github.com/jiduque/wordzee/scrabble_scrapper.py) (**repo doesn't exist yet**). In it, I gather the words allowed in Scrabble and remove the words longer than 7 letters and smaller than 2. In this notebook, I will just import the result.

In [1]:
from scrabble_scrapper import load_words

VALID_WORDS = load_words()

### Letter Values
I took screenshots of several games so that I can get the points of the letters at different rounds. The pattern I noticed was that the points of the letters are multiplied by the round number. So if "B" was worth 20 points in round five, it was worth 4 in round one. This makes it easier to get the values because we don't need to look at all letters in each round, we could do it in a couple of games. Below are some observations I made and a function to normalize them to what their values would be in the first round.

In [2]:
from typing import Union
from string import ascii_lowercase

# letter, points, round
DataPoint = tuple[str, int, int]
Data = list[DataPoint]
LetterVals = dict[str, int]

letter_data = [
    ("a", 2, 2), ("b", 20, 5), ("c", 12, 3), ("d", 2, 1), ("e", 1, 1), ("f", 4, 1),
    ("g", 6, 2), ("h", 8, 2), ("i", 1, 1), ("j", 30, 3), ("k", 5, 1), ("l", 3, 3),
    ("m", 6, 2), ("n", 1, 1), ("o", 1, 1), ("p", 20, 5), ("q", 30, 3), ("r", 2, 2),
    ("s", 2, 2), ("t", 1, 1), ("u", 2, 1), ("v", 16, 4), ("w", 16, 4),  ("x", 8, 1),
    ("y", 8, 2), ("z", 50, 5)
]

def letter_values(data: Data) -> Union[LetterVals, None]:
    output = {}
    for letter, point, game_round in data:
        x = point // game_round
        if (letter in output) and (output[letter] != x):
            print("inconsistency present")
            return None
        output[letter] = x

    diff = set(ascii_lowercase) - set(output.keys())
    if diff:
        print(f"letters are missing in the data: {diff}")

    return output


LETTER_VALUES = letter_values(letter_data)
LETTER_VALUES[' '] = 0

## The Perfect Game

Let's be dreamers and look for the perfect game: the set of words and board configuration with the highest score. The obvious way to go about that is to iterate through all board configurations and finding the collection of words that gives the most points. But before just jumping right in, we should calculate how many possible games there are to make sure we don't fry our computer.

### A Little Combinatorics
At a high level, the number we're looking for is the total number of board configurations times the total number of possible valid collections of words. The number of board configurations is equal to the number of ways we can arrange the triple word and double word bonuses, ${5}\choose{2}$, times the number of places we can place those bonuses. For the first three rows that is $R$ and for the last two it is $R-1$. So, total board configurations can be computed with the code below.

In [3]:
from math import comb

where_tl_go = comb(5, 2)
where_bonus_goes = 3 * 4 * 5 * 5 * 6
board_configurations = where_tl_go * where_bonus_goes

The number of valid word collections is pretty much: $\prod_{i=3}^7 #(\text{words \leq i})$. The code block below will compute that value and summarize the values of interest.

In [4]:
from operator import mul
from collections import Counter
from itertools import accumulate
from functools import reduce

counter = Counter(map(len, VALID_WORDS))
cdf = list(accumulate(counter[i] for i in range(7 + 1)))
sets_of_words = reduce(mul, cdf[2:])

possible_games = sets_of_words * board_configurations
print(f"{board_configurations=}\nnumber_of_words={len(VALID_WORDS)}\n{sets_of_words=:e}\n{possible_games=:e}")

board_configurations=18000
number_of_words=8546
sets_of_words=9.850437e+19
possible_games=1.773079e+24


<p align="center">
    <img src="https://tenor.com/view/delusions-preston-teagardin-robert-pattinson-the-devil-all-the-time-upset-gif-18149886.gif" />
<p/>

Well, that's not going to happen; that search space is way too large. I would die before this finished running on my computer. We need something more clever because I can't die before being the world's best Wordzee player.

## Words With Largest Expected Value
Let $R$ be the length of a row. Then for rows 1, 2, and 3 we get the following expected value:

### First Three Rows

$$
E\left[ \text{word} | \text{R} \right] = \frac{5R + 7}{5 R} \sum_i^R L_i
$$

where $L_i$ is the number of points the $i$th letter of the word gets. This also assumes the length of the word is equal to or less than the row size. If the word is longer than the row size, the value is 0. To calculate this, we use a fan-favorite: the law of total expectation. Given that we know the bonus amount $B$ and that the location is uniformly distributed, the expectation can be easily calculated. There are $R$ cases of where the bonus is located, each with probability $1/R$ of occurring. Each letter value will be added normally $R-1$ times and one time its values will be multiplied by $B$. This means given $B = b$, the expected value of a word in a row is:


$$
E\left[ \text{word} | \text{R}, B=b \right] = \frac{R - 1 + b}{R} \sum_i^R L_i
$$

Now we will use the law of total expectation to get the final result.

$$
E\left[ \text{word} | \text{R} \right] = \sum_{b=2}^3 E\left[ \text{word} | \text{R,} B = b \right] P\left( B = b \right)
$$

$$
= \frac{2}{5} E\left[ \text{word} | \text{R,} B = 3 \right] + \frac{3}{5} E\left[ \text{word} | \text{R,} B = 2 \right]
$$

$$
= \frac{2(R + 2)}{5R} \sum_i^R L_i +  \frac{3(R + 1)}{5R} \sum_i^R L_i
$$

Simplifying that last expression gives us the result shown at the beginning of this section.

### Last Two Rows
For the last two rows, the last letter is reserved for the word bonuses. This means we can only give a letter bonus to all but the last one. With regard to our calculation, this means that the expected value computed above can be used for the first $R-1$ letters, and the value of the last letter will just be added to the total.  The last thing to add is the word bonus. Putting that all together we get:

$$

E\left[ \text{word} | \text{R} \right] = \left[L_R + \frac{5R + 2}{5(R - 1)} \sum_i^{R-1} L_i\right] M(\text{word}, R)

$$

Where $M$ is defined as the following conditional:

$$M(\text{word}, R) = \left\{
    \begin{array}{lr}
        1 & \text{length of word } < R\\
        2 &   R = 6\\
        3 &   R = 7\\
    \end{array}\right.
$$

### All Together

In [5]:
Word = str

def expected(word: Word, r: int, letter_vals: LetterVals) -> float:
    if len(word) > r:
        return 0

    last_val = letter_vals[word[-1]] if r > 5 else 0
    letters_to_analyze = word[:-1] if r > 5 else word
    row_length = r - 1 if r > 5 else r
    output = last_val + all_random_expected(letters_to_analyze, row_length, letter_vals)

    return multiplier(word, r) * output


def all_random_expected(word: Word, r: int, letter_vals: LetterVals) -> float:
    if len(word) > r:
        return 0
    output = sum(letter_vals[letter] for letter in word)
    output *= (5 * r + 7) / (5 * r)
    return output

def multiplier(word: Word, r: int) -> int:
    if len(word) < r:
        return 1
    return r - 4

### Results

In [6]:
Words = list[Word]

def top_k_for_row(words: Words, letter_vals: LetterVals, row: int, k: int = 5) -> Words:
    r = row + 3
    output = list(map(lambda w: (w, expected(w, r, letter_vals)), words))
    output.sort(key=lambda x: x[1], reverse=True)
    return output[:k]


def top_k(words: Words, letter_vals: LetterVals, k: int = 5) -> list[Words]:
    return list(map(
        lambda row: top_k_for_row(words, letter_vals, row, k),
        range(5)
    ))


for output_row, vals in enumerate(top_k(VALID_WORDS, LETTER_VALUES)):
    print(f"row: {output_row+1}")
    print("\t" + "\t\t".join(map(lambda x: f"{x[0]}: {x[1]:.2f}", vals)))

row: 1
	jo: 16.13		qi: 16.13		za: 16.13		xu: 14.67		ax: 13.20
row: 2
	zzz: 40.50		zuz: 29.70		zax: 25.65		cuz: 21.60		pyx: 21.60
row: 3
	jazzy: 44.80		qajaq: 40.96		jazz: 39.68		zizz: 39.68		zzz: 38.40
row: 4
	jazzbo: 91.60		jazzed: 85.92		zizzed: 85.92		jazzer: 83.92		jazzes: 83.92
row: 5
	zigzags: 106.60		palazzi: 102.90		palazzo: 102.90		jackdaw: 97.10		jackpot: 95.50


### Greedy Thoughts

The information above was nice, but it's pretty useless in a game because those words are only useful if we don't know the board configurations. We can get around this by thinking about the problem a little more instead of just jumping in. The best way to think about this problem is to think about it in a greedy way. To maximize a game, we need to maximize each row (and possibly try to get the extra wordzee bonus for filling up the whole board). This means for each row, we should look for potentially two words: the max one and the max one that fills the whole row. In the brute force way, the real killer in this search is the number of board configurations. Fortunately, each word has the best configuration when the letter bonus is in the location of the word's letter with the most points. We no longer need to search through configurations and can just search through the words for each row. To do this, I built a data structure that has two running max's for each row. Then we can just feed each word to this data structure and then retrieve the best words and the location of the best letter.
All of this logic is written in the data structures found [here](https://www.github.com/jiduque/wordzee/wordzee_search.py). For this notebook, I will just import it and do the final steps manually.

In [7]:
from wordzee_search import GameAnalyzer

analyzer = GameAnalyzer(LETTER_VALUES)
for word_str in VALID_WORDS:
    analyzer.update(word_str)

### Results

In [8]:
for ra in analyzer.row_analyzers:
    print(f"Row {ra.row}:")
    print(f"\tbest:{ra.best}")
    if ra.does_it_matter():
        print(f"\tfull: {ra.best_full}")

Row 1:
	best:Word(word='zzz', base=30, max_loc=0, points_at_max_loc=10)
Row 2:
	best:Word(word='jazz', base=31, max_loc=0, points_at_max_loc=10), Word(word='zizz', base=31, max_loc=0, points_at_max_loc=10)
Row 3:
	best:Word(word='jazzy', base=35, max_loc=0, points_at_max_loc=10)
Row 4:
	best:Word(word='jazzbo', base=36, max_loc=0, points_at_max_loc=10)
Row 5:
	best:Word(word='jazzbo', base=36, max_loc=0, points_at_max_loc=10)
	full: Word(word='zigzags', base=29, max_loc=0, points_at_max_loc=10)


Only row 5 had a word that maximizes points but doesn't fill it. This means we have two scenarios to check for the best game: the one where we just maximize word and then one where we fill the board. We calculate the score to see which one is the better one.

In [9]:
def game_points(words: Words, bonuses: list[int], orders: list[int]) -> int:
    total_points, total_slots = 0, 0
    all_slots = 25
    for i in range(5):
        word = words[i]
        order = orders[i]
        b = bonuses[i]
        points = order * word.points_with_bonus(b)
        if order == 4 and word.length == 6:
            points *= 2
        elif order == 5 and word.length == 7:
            points *= 3

        total_points += points
        total_slots += word.length

    if total_slots == all_slots:
        total_points += 100

    return total_points

When it comes to order and triple-letter bonus locations: we take a greedy approach again. Take the ones that will maximize everything. From the look of the scores, that means we play the words in row order and then the last two rows have the triple-letter bonuses.

In [10]:
greedy_words = [ra.best.words[0] for ra in analyzer.row_analyzers]
game_points(greedy_words, [2, 2, 2, 3, 3], [1, 2, 3, 4, 5])

985

In [11]:
filling_words = [ra.best.words[0] for ra in analyzer.row_analyzers]
filling_words[4] = analyzer.row_analyzers[4].best_full.words[0]
game_points(filling_words, [2, 2, 2, 3, 3], [1, 2, 3, 4, 5])

1540

## Best Game

This means there are two games tied for the best games you could have. This is because the second row could be 'jazz' or 'zizz' and it wouldn't make a difference. The board configuration is the following:

- triple letter bonuses are in the last two rows
- all bonus locations are on the first character.

Strategy wise, you will fill each row in order with the following words:
- zzz
- jazz or zizz
- jazzy
- jazzbo
- zigzags

Because all the slots are filled, you get all bonuses. Coming out to a whopping 1540 points. This makes the best two games nearly twice the number of points as the third-best game, 985. That game is the same configuration and same strategy, but using jazzbo for the last row as well as the fourth. I will compare strategies in another post soon-ish.